In [6]:
!pip install requests beautifulsoup4 tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00


In [4]:
import requests
from bs4 import BeautifulSoup

def get_article_text(url):
    # Fetch the HTML content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Parse the content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the article text. This step may vary depending on the website's structure.
    # Here's a general approach that extracts all the text within paragraph tags.
    paragraphs = soup.find_all('p')
    article_text = " ".join(p.text for p in paragraphs)

    return article_text

def get_article_title(url):
    # Fetch the HTML content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Parse the content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the article text. This step may vary depending on the website's structure.
    # Here's a general approach that extracts all the text within paragraph tags.
    titles = soup.find_all('title')
    article_title = " ".join(tit.text for tit in titles)

    return article_title
url = "https://vitalik.ca/general/2023/08/16/communitynotes.html"
text = get_article_text(url)
#print(text)
title = get_article_title(url)

In [5]:
title

' What do I think about Community Notes? '

In [8]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

def token_len(txt):
  tokens = tokenizer.encode(txt, disallowed_special=())
  return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=0,
    length_function=token_len,
    separators=["\n\n", "\n", " ", ""]
)
chunks = text_splitter.split_text(text)

In [11]:
data = []
for chunk in chunks:
  data.append({'text': chunk})

In [14]:
import json
with open("data.jsonl", "w") as file:
    for entry in data:
        json_str = json.dumps(entry)
        file.write(json_str + "\n")